In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import pickle
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection as ms
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold

rand_st=4

In [2]:
output=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx')
errors=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx')

In [3]:
elos_1=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos1.csv')
elos_2=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos2.csv')
elos_3=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos3.csv')
elos_4=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos4.csv')
features_1=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features1.csv')
features_3=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features3.csv')
features_2=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features2.csv')
features_4=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features4.csv')

In [4]:
X_train_mean_1, X_test_mean_1, Y_train_mean_1, Y_test_mean_1 =ms.train_test_split(features_1, elos_1['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_1, X_test_diff_1, Y_train_diff_1, Y_test_diff_1 =ms.train_test_split(features_1, elos_1['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_2, X_test_mean_2, Y_train_mean_2, Y_test_mean_2 =ms.train_test_split(features_2, elos_2['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_2, X_test_diff_2, Y_train_diff_2, Y_test_diff_2 =ms.train_test_split(features_2, elos_2['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_3, X_test_mean_3, Y_train_mean_3, Y_test_mean_3 =ms.train_test_split(features_3, elos_3['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_3, X_test_diff_3, Y_train_diff_3, Y_test_diff_3 =ms.train_test_split(features_3, elos_3['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_4, X_test_mean_4, Y_train_mean_4, Y_test_mean_4 =ms.train_test_split(features_4, elos_4['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_4, X_test_diff_4, Y_train_diff_4, Y_test_diff_4 =ms.train_test_split(features_4, elos_4['DiffElos'], test_size = 0.3,random_state=rand_st)

In [5]:
model_mean=RandomForestRegressor(bootstrap=False, criterion='mae', max_depth=None,
           max_features=2, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=5, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=-1, oob_score=False, random_state=4,
           verbose=0, warm_start=False)

model_diff=RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=None,
           max_features=1, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=5,
           min_samples_split=25, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=-1, oob_score=True, random_state=4,
           verbose=0, warm_start=False)

In [6]:
def find_mean_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
        count+=2
    return round(er/count,3)

def find_abs_error(pred,elos):
    er=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
    return round(er,3)

def find_root_mean_squared_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=(elos.iloc[ind,0]-val[0])**2
        er+=(elos.iloc[ind,1]-val[1])**2
        count+=2
    return round((er/count)**0.5,3)

In [7]:
def predictions_report(pred_target,predictions):
    mae_report=mae(pred_target,predictions)
    print ('MAE = {:.3f}'.format(mae_report))
    
    rmse_report = (mse(pred_target, predictions))**0.5
    print ('RMSE = {:.3f}'.format(rmse_report))
    
    corr_coef = pearsonr(pred_target,predictions)
    print("Correlation coefficient = {:.3f}".format(corr_coef[0]))
    
    return (mae_report,rmse_report,corr_coef)

### Подбор модели для кластера 1

In [8]:
start1 = time.time()

rf1=model_mean.fit(X_train_mean_1,Y_train_mean_1)
rf_pred1=rf1.predict(X_test_mean_1)
scores1=predictions_report(Y_test_mean_1,rf_pred1)
full_time1=round(time.time() - start1,3)
print(full_time1, 'c')

MAE = 106.925
RMSE = 132.043
Correlation coefficient = 0.270
15.508 c


In [9]:
output = output.append(pd.Series([str(model_mean),
                                 scores1[0],scores1[1], 
                                 scores1[2][0],full_time1,'Обучение на среднем ЭЛО по 1 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.980777,129.002684,0.310189,0.026,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",173.676816,218.061005,0.564917,0.032,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",77.387126,95.797568,0.217573,0.040,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",108.608625,135.853982,0.501589,0.039,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.616755,94.974990,0.237813,0.040,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.869006,132.827010,0.390109,0.028,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.319818,94.829941,0.228683,0.028,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",103.741893,134.630404,0.404880,0.037,Обучение на разнице в ЭЛО по 4 кластеру
8,"RandomForestRegressor(bootstrap=False, criteri...",106.924601,132.042952,0.269526,15.508,Обучение на среднем ЭЛО по 1 кластеру


In [10]:
start2 = time.time()

rf2=model_diff.fit(X_train_diff_1,Y_train_diff_1)
rf_pred2=rf2.predict(X_test_mean_1)
scores2=predictions_report(Y_test_diff_1,rf_pred2)
full_time2=round(time.time() - start2,3)
print(full_time2, 'c')

MAE = 188.770
RMSE = 229.315
Correlation coefficient = 0.527
7.519 c


In [11]:
output = output.append(pd.Series([str(model_diff),
                                 scores2[0],scores2[1], 
                                 scores2[2][0],full_time2,'Обучение на разнице ЭЛО по 1 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.980777,129.002684,0.310189,0.026,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",173.676816,218.061005,0.564917,0.032,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",77.387126,95.797568,0.217573,0.040,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",108.608625,135.853982,0.501589,0.039,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.616755,94.974990,0.237813,0.040,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.869006,132.827010,0.390109,0.028,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.319818,94.829941,0.228683,0.028,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",103.741893,134.630404,0.404880,0.037,Обучение на разнице в ЭЛО по 4 кластеру
8,"RandomForestRegressor(bootstrap=False, criteri...",106.924601,132.042952,0.269526,15.508,Обучение на среднем ЭЛО по 1 кластеру
9,"RandomForestRegressor(bootstrap=True, criterio...",188.770360,229.315473,0.527370,7.519,Обучение на разнице ЭЛО по 1 кластеру


In [12]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [13]:
white_elos_1 = rf_pred1 + rf_pred2/2
black_elos_1 = rf_pred1-rf_pred2/2
pred_elos1=list(zip(white_elos_1,black_elos_1))
print(len(pred_elos1))
pred_elos1[:5]

1279


[(1891.883, 1796.497),
 (1881.2620000000002, 1842.328),
 (1920.93, 1840.675),
 (1852.911, 1775.859),
 (1812.0900000000001, 1888.27)]

In [14]:
find_abs_error(pred_elos1,elos_1),find_mean_error(pred_elos1,elos_1),find_root_mean_squared_error(pred_elos1,elos_1)

(404246.543, 158.032, 195.802)

In [15]:
inner_mae_1=find_mean_error(pred_elos1,elos_1)

In [16]:
errors =errors.append(pd.Series(['1 кластер','Random Forest',
                  find_abs_error(pred_elos1,elos_1),
                  find_mean_error(pred_elos1,elos_1),
                   find_root_mean_squared_error(pred_elos1,elos_1) ],
                                   index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,1.150193e+06,134.936,168.90800
1,2 кластер,LinearRegression,1.566282e+06,95.284,117.43700
2,3 кластер,LinearRegression,1.127526e+06,94.559,115.89300
3,4 кластер,LinearRegression,1.099642e+06,94.601,116.29500
4,Общая ошибка,LinearRegression,1.235911e+06,104.845,129.63325
5,1 кластер,Random Forest,4.042465e+05,158.032,195.80200


In [17]:
errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)

### Подбор модели для 2 кластера

In [18]:
start3 = time.time()

rf3=model_mean.fit(X_train_mean_2,Y_train_mean_2)
rf_pred3=rf3.predict(X_test_mean_2)
scores3=predictions_report(Y_test_mean_2,rf_pred3)
full_time3=round(time.time() - start3,3)
print(full_time3, 'c')

MAE = 77.402
RMSE = 94.900
Correlation coefficient = 0.197
35.547 c


In [19]:
output = output.append(pd.Series([str(model_mean),
                                 scores3[0],scores3[1], 
                                 scores3[2][0],full_time3,'Обучение на среднем ЭЛО по 2 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.980777,129.002684,0.310189,0.026,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",173.676816,218.061005,0.564917,0.032,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",77.387126,95.797568,0.217573,0.040,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",108.608625,135.853982,0.501589,0.039,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.616755,94.974990,0.237813,0.040,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.869006,132.827010,0.390109,0.028,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.319818,94.829941,0.228683,0.028,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",103.741893,134.630404,0.404880,0.037,Обучение на разнице в ЭЛО по 4 кластеру
8,"RandomForestRegressor(bootstrap=False, criteri...",106.924601,132.042952,0.269526,15.508,Обучение на среднем ЭЛО по 1 кластеру
9,"RandomForestRegressor(bootstrap=True, criterio...",188.770360,229.315473,0.527370,7.519,Обучение на разнице ЭЛО по 1 кластеру


In [20]:
start4 = time.time()

rf4=model_diff.fit(X_train_diff_2,Y_train_diff_2)
rf_pred4=rf4.predict(X_test_mean_2)
scores4=predictions_report(Y_test_diff_2,rf_pred4)
full_time4=round(time.time() - start4,3)
print(full_time4, 'c')

MAE = 114.877
RMSE = 142.274
Correlation coefficient = 0.464
24.118 c


In [21]:
output = output.append(pd.Series([str(model_diff),
                                 scores4[0],scores4[1], 
                                 scores4[2][0],full_time4,'Обучение на разнице ЭЛО по 2 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.980777,129.002684,0.310189,0.026,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",173.676816,218.061005,0.564917,0.032,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",77.387126,95.797568,0.217573,0.040,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",108.608625,135.853982,0.501589,0.039,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.616755,94.974990,0.237813,0.040,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.869006,132.827010,0.390109,0.028,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.319818,94.829941,0.228683,0.028,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",103.741893,134.630404,0.404880,0.037,Обучение на разнице в ЭЛО по 4 кластеру
8,"RandomForestRegressor(bootstrap=False, criteri...",106.924601,132.042952,0.269526,15.508,Обучение на среднем ЭЛО по 1 кластеру
9,"RandomForestRegressor(bootstrap=True, criterio...",188.770360,229.315473,0.527370,7.519,Обучение на разнице ЭЛО по 1 кластеру


In [22]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [23]:
white_elos_2 = rf_pred3 + rf_pred3/2
black_elos_2 = rf_pred3-rf_pred4/2
pred_elos2=list(zip(white_elos_2,black_elos_2))
print(len(pred_elos2))
pred_elos2[:5]

2466


[(3729.5924999999997, 2447.691),
 (3798.1575000000003, 2539.47),
 (3725.9325, 2449.5955),
 (3757.54875, 2498.269),
 (3730.45125, 2528.6895000000004)]

In [24]:
find_abs_error(pred_elos2,elos_2),find_mean_error(pred_elos2,elos_2),find_root_mean_squared_error(pred_elos2,elos_2)

(3344622.109, 678.147, 893.174)

In [25]:
inner_mae_2=find_mean_error(pred_elos2,elos_2)

In [26]:
errors =errors.append(pd.Series(['2 кластер','Random Forest',
                        find_abs_error(pred_elos2,elos_2),
                        find_mean_error(pred_elos2,elos_2),
                        find_root_mean_squared_error(pred_elos2,elos_2)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,1.150193e+06,134.936,168.90800
1,2 кластер,LinearRegression,1.566282e+06,95.284,117.43700
2,3 кластер,LinearRegression,1.127526e+06,94.559,115.89300
3,4 кластер,LinearRegression,1.099642e+06,94.601,116.29500
4,Общая ошибка,LinearRegression,1.235911e+06,104.845,129.63325
5,1 кластер,Random Forest,4.042465e+05,158.032,195.80200
6,2 кластер,Random Forest,3.344622e+06,678.147,893.17400


### Подбор модели для 3 кластера

In [27]:
start5 = time.time()

rf5=model_mean.fit(X_train_mean_3,Y_train_mean_3)
rf_pred5=rf5.predict(X_test_mean_3)
scores5=predictions_report(Y_test_mean_3,rf_pred5)
full_time5=round(time.time() - start5,3)
print(full_time5, 'c')

MAE = 82.135
RMSE = 96.558
Correlation coefficient = 0.149
25.474 c


In [28]:
output = output.append(pd.Series([str(model_mean),
                                 scores5[0],scores5[1], 
                                 scores5[2][0],full_time5,'Обучение на среднем ЭЛО по 3 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.980777,129.002684,0.310189,0.026,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",173.676816,218.061005,0.564917,0.032,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",77.387126,95.797568,0.217573,0.040,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",108.608625,135.853982,0.501589,0.039,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.616755,94.974990,0.237813,0.040,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.869006,132.827010,0.390109,0.028,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.319818,94.829941,0.228683,0.028,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",103.741893,134.630404,0.404880,0.037,Обучение на разнице в ЭЛО по 4 кластеру
8,"RandomForestRegressor(bootstrap=False, criteri...",106.924601,132.042952,0.269526,15.508,Обучение на среднем ЭЛО по 1 кластеру
9,"RandomForestRegressor(bootstrap=True, criterio...",188.770360,229.315473,0.527370,7.519,Обучение на разнице ЭЛО по 1 кластеру


In [29]:
start6 = time.time()

rf6=model_diff.fit(X_train_diff_3,Y_train_diff_3)
rf_pred6=rf6.predict(X_test_mean_3)
scores6=predictions_report(Y_test_diff_3,rf_pred6)
full_time6=round(time.time() - start6,3)
print(full_time6, 'c')

MAE = 105.326
RMSE = 137.164
Correlation coefficient = 0.368
13.734 c


In [30]:
output = output.append(pd.Series([str(model_diff),
                                 scores6[0],scores6[1], 
                                 scores6[2][0],full_time6,'Обучение на разнице ЭЛО по 3 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.980777,129.002684,0.310189,0.026,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",173.676816,218.061005,0.564917,0.032,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",77.387126,95.797568,0.217573,0.040,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",108.608625,135.853982,0.501589,0.039,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.616755,94.974990,0.237813,0.040,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.869006,132.827010,0.390109,0.028,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.319818,94.829941,0.228683,0.028,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",103.741893,134.630404,0.404880,0.037,Обучение на разнице в ЭЛО по 4 кластеру
8,"RandomForestRegressor(bootstrap=False, criteri...",106.924601,132.042952,0.269526,15.508,Обучение на среднем ЭЛО по 1 кластеру
9,"RandomForestRegressor(bootstrap=True, criterio...",188.770360,229.315473,0.527370,7.519,Обучение на разнице ЭЛО по 1 кластеру


In [31]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [32]:
white_elos_3 = rf_pred5 + rf_pred6/2
black_elos_3 = rf_pred5-rf_pred6/2
pred_elos3=list(zip(white_elos_3,black_elos_3))
print(len(pred_elos3))
pred_elos3[:5]

1789


[(2297.04, 2086.99),
 (2312.1745, 2083.9555),
 (2328.017, 2119.808),
 (2336.3205, 2110.5495),
 (2302.3575, 2070.6525)]

In [33]:
find_abs_error(pred_elos3,elos_3),find_mean_error(pred_elos3,elos_3),find_root_mean_squared_error(pred_elos3,elos_3)

(357067.155, 99.795, 122.906)

In [34]:
inner_mae_3=find_mean_error(pred_elos3,elos_3)

In [ ]:
errors =errors.append(pd.Series(['3 кластер','Random Forest',
                                 find_abs_error(pred_elos3,elos_3),
                                 find_mean_error(pred_elos3,elos_3),
                                 find_root_mean_squared_error(pred_elos3,elos_3)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,1.150193e+06,134.936,168.90800
1,2 кластер,LinearRegression,1.566282e+06,95.284,117.43700
2,3 кластер,LinearRegression,1.127526e+06,94.559,115.89300
3,4 кластер,LinearRegression,1.099642e+06,94.601,116.29500
4,Общая ошибка,LinearRegression,1.235911e+06,104.845,129.63325
5,1 кластер,Random Forest,4.042465e+05,158.032,195.80200
6,2 кластер,Random Forest,3.344622e+06,678.147,893.17400
7,3 кластер,Random Forest,3.570672e+05,99.795,122.90600


### Подбор модели для 4 кластера

In [ ]:
start7 = time.time()

rf7=model_mean.fit(X_train_mean_4,Y_train_mean_4)
rf_pred7=rf7.predict(X_test_mean_4)
scores7=predictions_report(Y_test_mean_4,rf_pred7)
full_time7=round(time.time() - start7,3)
print(full_time7, 'c')

In [ ]:
output = output.append(pd.Series([str(model_mean),
                                 scores7[0],scores7[1], 
                                 scores7[2][0],full_time7,'Обучение на среднем ЭЛО по 4 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

In [ ]:
start8 = time.time()

rf8=model_diff.fit(X_train_diff_4,Y_train_diff_4)
rf_pred8=rf8.predict(X_test_mean_4)
scores8=predictions_report(Y_test_diff_4,rf_pred8)
full_time8=round(time.time() - start8,3)
print(full_time8, 'c')

In [ ]:
output = output.append(pd.Series([str(model_diff),
                                 scores8[0],scores8[1], 
                                 scores8[2][0],full_time8,'Обучение на разнице ЭЛО по 4 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

In [ ]:
white_elos_4 = rf_pred7 + rf_pred8/2
black_elos_4 = rf_pred7-rf_pred8/2
pred_elos4=list(zip(white_elos_4,black_elos_4))
print(len(pred_elos4))
pred_elos4[:5]

In [ ]:
find_abs_error(pred_elos4,elos_4),find_mean_error(pred_elos4,elos_4),find_root_mean_squared_error(pred_elos4,elos_4)

In [ ]:
inner_mae_4=find_mean_error(pred_elos4,elos_4)

In [ ]:
errors =errors.append(pd.Series(['4 кластер','Random Forest',find_abs_error(pred_elos4,elos_4),
                                 find_mean_error(pred_elos4,elos_4),
                                 find_root_mean_squared_error(pred_elos4,elos_4)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

In [ ]:
rf_mae=(inner_mae_4+inner_mae_3+inner_mae_2+inner_mae_1)/4
rf_mae

In [ ]:
rf_abs=(find_abs_error(pred_elos1,elos_1)+
        find_abs_error(pred_elos2,elos_2)+
        find_abs_error(pred_elos3,elos_3)+
        find_abs_error(pred_elos4,elos_4))/4
rf_abs

In [ ]:
rf_rmse=(find_root_mean_squared_error(pred_elos1,elos_1)+
         find_root_mean_squared_error(pred_elos2,elos_2)+
         find_root_mean_squared_error(pred_elos3,elos_3)+
         find_root_mean_squared_error(pred_elos4,elos_4))/4
rf_rmse

In [ ]:
errors = errors.append(pd.Series(['Общая ошибка','Random Forest',
                  rf_abs,
                  rf_mae,
                 rf_rmse],index=errors.columns) ,ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors